In [10]:
import re

from abc import abstractmethod

class Instruction():
    
    def __init__(self, value):
        self.value = value
    
    @abstractmethod
    def apply(self, memory, mask):
        pass
    
class MemWriteInstruction1(Instruction):
    
    def __init__(self, value):
        super(MemWriteInstruction1, self).__init__(value)
        self.memoryaddr = int(self.value[0])
        self.writeval = int(self.value[1])
        
    def apply(self, memory, mask):
        valuestring = "0b"
        for idx, bitstring in enumerate(mask):
            if bitstring == 'X':
                valuestring += str((self.writeval & (1 << (len(mask) - idx - 1))) >> (len(mask) - idx - 1))
            else:
                valuestring += bitstring
        value = int(valuestring, 2)
        memory[self.memoryaddr] = value
        return memory, mask
    
class MaskChangeInstruction(Instruction):
    
    def __init__(self, value):
        super(MaskChangeInstruction, self).__init__(value)
        self.mask = self.value
        
    def apply(self, memory, mask):
        return memory, self.mask

class Computer():
    
    def __init__(self):
        self.memory = dict()
        self.mask = None
    
    def _apply(self, instruction):
        self.memory, self.mask = instruction.apply(self.memory, self.mask)
        
    def run(self, instructionset):
        for instruction in instructionset:
            self._apply(instruction)
        return self.memory, self.mask

maskregex = r"mask=(?P<mask>[X|1|0]{36})"
memregex = r"mem\[(?P<addr>\d+)\]=(?P<val>\d+)"

    
def build1(stringrep):
    if re.match(maskregex, stringrep):
        mask = re.match(maskregex, stringrep).group("mask")
        return MaskChangeInstruction(mask)
    elif re.match(memregex, stringrep):
        memaddr = re.match(memregex, stringrep).group("addr")
        memval = re.match(memregex, stringrep).group("val")
        return MemWriteInstruction1((memaddr, memval))

filename = "./input.txt"
with open(filename) as f:
    content = [line.strip() for line in f.readlines()]
    content = ["".join(line.split()) for line in content]

computer = Computer()
instructionset = [build1(rep) for rep in content]
memory, _ = computer.run(instructionset)
sum(memory.values())

6513443633260

In [11]:
#'Cause I really like trees.
class MemoryRepresentationTree():
        
        def __init__(self, representation):
            self.representation = representation
            self.children = []
            for idx, char in enumerate(representation):
                if char == 'X':
                    for possiblevalue in ['0','1']:
                        child = MemoryRepresentationTree(representation[:idx] + possiblevalue + representation[idx + 1:])
                        self.children.append(child)
                    break;
            
        def collect(self):
            if self.isleaf():
                return [self.representation]
            return [r for representations in [child.collect() for child in self.children] for r in representations]
            
        def isleaf(self):
            return not self.children

class MemWriteInstruction2(Instruction):
    
    def __init__(self, value):
        super(MemWriteInstruction2, self).__init__(value)
        self.maskaddr = int(self.value[0])
        self.writeval = int(self.value[1])   
        
    def apply(self, memory, mask):
        memrep = self._buildmemrep(mask)
        memreptree = MemoryRepresentationTree(memrep)
        for rep in memreptree.collect():
            memory[int(rep, 2)] = self.writeval
        return memory, mask
    
    def _buildmemrep(self, mask): 
        valuestring = "0b"
        for idx, bitstring in enumerate(mask):
            if bitstring == '0':
                valuestring += str((self.maskaddr & (1 << (len(mask) - idx - 1))) >> (len(mask) - idx - 1))
            elif bitstring == '1':
                valuestring += '1'
            elif bitstring == 'X':
                valuestring += 'X'
        return valuestring
        

def build2(stringrep):
    if re.match(maskregex, stringrep):
        mask = re.match(maskregex, stringrep).group("mask")
        return MaskChangeInstruction(mask)
    elif re.match(memregex, stringrep):
        memaddr = re.match(memregex, stringrep).group("addr")
        memval = re.match(memregex, stringrep).group("val")
        return MemWriteInstruction2((memaddr, memval))
    
computer = Computer()
instructionset = [build2(rep) for rep in content]
memory, _ = computer.run(instructionset)
sum(memory.values())

3442819875191